In [ ]:
import pandas as pd
from datetime import date, datetime

from apscheduler.schedulers.background import BackgroundScheduler

In [ ]:
def run_alert_log_background():
    ALERTED = set()
    sched = BackgroundScheduler()
    sched.add_job(send_alert_text, 'cron', day_of_week='mon-fri', hour='7-16', second='*/10',args=[ALERTED] )
    sched.start()

In [ ]:
def send_alert_text(alerted):
    alert_log = get_todays_file()
    alerts = unique_alert(alerted, alert_log)
    for alert in alerts:
        send_sms(alert)
        print(alert)
    if len(alerts) < 1:
        print("no new alerts found {}".format(datetime.now())) 

In [ ]:
def get_todays_file():
    today = date.today()
    # file_path = "alertlogging.Running Up.{}.csv".format(today.strftime("%Y%m%d"))
    file_path = "alertlogging.Running Up.20221129.csv" # using this path to use included sample file
    try:
        alert_log = pd.read_csv(file_path)
    except FileNotFoundError:
        alert_log = []
        print("No file yet")
    return alert_log

In [ ]:
def unique_alert(alerted, alert_log):
    symbol_alerts = []
    if len(alert_log) > 0:
        firsts = alert_log.groupby('Symbol', as_index=False).first()
        alerted_reset = reset_alerted_daily(alerted)
        for symbol in firsts.Symbol:
            if symbol not in alerted_reset:
                alerted_reset.add(symbol)
                symbol_alert = firsts.loc[firsts.Symbol == symbol][['Symbol', 'TimeStamp', 'Price', 'Volume Today']]
                symbol_alerts.append(symbol_alert)
    return symbol_alerts

In [ ]:
def reset_alerted_daily(alerted):
    now = datetime.now()
    if ( now.hour == 7 and now.minute == 1 and now.second == 1 ) :
        alerted = set()
    return alerted

In [ ]:
import email, smtplib, ssl
from providers import PROVIDERS

def send_sms_via_email(
    number: str,
    message: str,
    provider: str,
    sender_credentials: tuple,
    subject: str = "sent using etext",
    smtp_server: str = "smtp.gmail.com",
    smtp_port: int = 465,
):
    sender_email, email_password = sender_credentials
    receiver_email = f'{number}@{PROVIDERS.get(provider).get("sms")}'

    email_message = f"Subject:{subject}\nTo:{receiver_email}\n{message}"

    with smtplib.SMTP_SSL(
        smtp_server, smtp_port, context=ssl.create_default_context()
    ) as email:
        email.login(sender_email, email_password)
        email.sendmail(sender_email, receiver_email, email_message)
        
def send_sms(message_str:str):
    number = "Your Phone number here (e.g. 1233211234)" #edit this string
    message = message_str
    provider = "your phone provider here(e.g. T-Mobile)" #edit this string
    sender_credentials = ("youremail@gmail.com", "SMTP forwarding password") #edit these two strings
    send_sms_via_email(number, message, provider, sender_credentials)

In [ ]:
run_alert_log_background()